## Installing and Importing necessary libraries

In [ ]:
!pip install fastapi uvicorn nest_asyncio ipywidgets requests

In [ ]:
# Installing libraries needed
!pip install fastapi uvicorn joblib pandas nest_asyncio

In [ ]:
# Import necessary libraries
import joblib
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import logging
import nest_asyncio
from uvicorn import run

In [ ]:
# Enable nesting of asyncio event loops for Jupyter compatibility
nest_asyncio.apply()

# Set up logging to output to both console and file
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) 


## Initializing FastAPI app

In [ ]:
# Creation of handlers
console_handler = logging.StreamHandler()  # To log to the console
file_handler = logging.FileHandler('app_logs.log')  # To log to a file

# Set logging levels for handlers
console_handler.setLevel(logging.DEBUG)  # Adjust based on your needs (e.g., DEBUG, INFO)
file_handler.setLevel(logging.DEBUG)


In [ ]:
# Creation of a formatter and attach it to handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [ ]:
# Initialize the FastAPI app
app = FastAPI()

## Loading of the ML Flow model

In [ ]:
# Load the XGBoost model and preprocessing pipeline
try:
    model_pipeline = joblib.load('random_forest_model_pipeline.pkl')
    logger.info("Model loaded successfully!")
    # Validate pipeline components
    assert 'preprocessor' in model_pipeline.named_steps, "Missing preprocessor in pipeline."
    assert 'model' in model_pipeline.named_steps, "Missing model in pipeline."
except Exception as e:
    logger.error(f"Failed to load model: {e}", exc_info=True)
    raise RuntimeError("Failed to load model pipeline. Ensure the .pkl file is valid and correctly configured.")


## Defining Data structure and defining the predict

In [ ]:
# Define input data structure
class CarData(BaseModel):
    Mileage: float
    EngineV: float
    Year: int
    Brand: str
    Body: str
    Engine_Type: str
    Registration: str
    Model: str

@app.post("/predict")
def predict(car_data: CarData):
    try:
        # Log raw input data
        logger.debug(f"Raw input data: {car_data.dict()}")

        # Convert input to DataFrame
        input_data = [[
            car_data.Mileage,
            car_data.EngineV,
            car_data.Year,
            car_data.Brand,
            car_data.Body,
            car_data.Engine_Type,
            car_data.Registration,
            car_data.Model
        ]]
        columns = ['Mileage', 'EngineV', 'Year', 'Brand', 'Body', 'Engine Type', 'Registration', 'Model']

        input_df = pd.DataFrame(input_data, columns=columns)
        logger.debug(f"Input DataFrame:\n{input_df}")

        # Preprocess input
        preprocessed_input = model_pipeline.named_steps['preprocessor'].transform(input_df)
        logger.debug(f"Preprocessed input:\n{preprocessed_input}")

        # Convert sparse matrix to dense
        preprocessed_input = preprocessed_input.toarray()
        logger.debug(f"Preprocessed dense input:\n{preprocessed_input}")

        # Make prediction
        predicted_price = model_pipeline.named_steps['model'].predict(preprocessed_input)[0]
        logger.info(f"Prediction: {predicted_price}")

        # Convert the numpy float to a standard Python float for serialization
        predicted_price = float(predicted_price)

        return {"prediction": predicted_price}

    except ValueError as ve:
        logger.error(f"ValueError during prediction: {ve}", exc_info=True)
        return {"error": f"Preprocessing or prediction failed. Error: {ve}"}

    except KeyError as ke:
        logger.error(f"KeyError: Missing expected column or key in input: {ke}", exc_info=True)
        return {"error": f"Input format error. Missing field: {ke}"}

    except Exception as e:
        logger.error(f"Unhandled error during prediction: {e}", exc_info=True)
        return {"error": f"Internal Server Error. Details: {e}"}


## Running FastAPI

In [ ]:
# Run the FastAPI app
run(app, host="127.0.0.1", port=8003)
